In [1]:
import pandas as pd

In [2]:
# read the log file
columns = ['IP','Cookie','User','Datetime','TMZ','Request','ReturnCode','Bytes','Referrer','UserAgent']
df = pd.read_csv('Log_file.log', header=None, names=columns, sep=' ')

In [3]:
df.head()

,IP,Cookie,User,Datetime,TMZ,Request,ReturnCode,Bytes,Referrer,UserAgent
0,193.87.12.30,-,-,[19/Feb/2020:06:25:50,+0100],GET /navbar/navbar-ukf.html HTTP/1.0,200,7584,-,-
1,193.87.12.30,-,-,[19/Feb/2020:06:25:55,+0100],GET /navbar/navbar-ukf.html HTTP/1.0,200,7584,-,-
2,193.87.12.30,-,-,[19/Feb/2020:06:25:56,+0100],GET /navbar/navbar-ukf.html HTTP/1.0,200,7584,-,-
3,193.87.12.30,-,-,[19/Feb/2020:06:25:57,+0100],GET /navbar/navbar-ukf.html HTTP/1.0,200,7584,-,-
4,193.87.12.30,-,-,[19/Feb/2020:06:25:49,+0100],GET / HTTP/1.1,200,20925,-,libwww-perl/6.08


# Data preprocessing

In [4]:
# data cleaning
suffix = (".jpg", ".jpeg",".png",".gif",".bmp",".css",".flv",".ico",".swf",".rss",".xml",".cur",".js",".json",".svg",
           ".woff",".eot",".font","POST","HEAD",".JPG")

for s in suffix:
    df.drop(df[df['Request'].str.contains(s)].index, inplace=True)


df.drop(df.loc[df.ReturnCode>=400].index, inplace=True)
df.drop(df.loc[df.ReturnCode<200].index, inplace=True)

In [5]:
robotstxt = df[df['Request'].str.contains("robots.txt")]
ips = robotstxt['IP']
print(ips.shape)

for ip in ips:
    df.drop(df[df['IP'].str.contains(ip)].index, inplace=True)

(333,)


In [6]:
df.drop(df[df['UserAgent'].str.contains('bot')].index, inplace=True)
df.drop(df[df['UserAgent'].str.contains('crawl')].index, inplace=True)
df.drop(df[df['UserAgent'].str.contains('spider')].index, inplace=True)

In [7]:
req_tab = pd.crosstab(index=df['Request'], columns="count")
req_tab = req_tab.sort_values(by=["count"], ascending=False)
req_tab.head(15)

col_0,count
Request,
GET /navbar/navbar-ukf.html HTTP/1.0,24854
GET / HTTP/1.1,5534
GET /index.php?option=com_acymailing&ctrl=cron HTTP/1.1,1440
GET /univerzita/kontakt/adresar-osob HTTP/1.1,495
GET /verejnost/aktuality/foto/image?view=image&format=raw&type=orig&id=16556 HTTP/1.1,319
GET /prijimacie-konanie/prihlasovanie-sa-na-studium HTTP/1.1,297
GET /verejnost/aktuality/foto/predaj20 HTTP/1.1,281
GET /images/oznamy/2020_Oznamy/Univerzita_Kon%C5%A1tant%C3%ADna_Filozofa_-_ponuka_na_predaj_auta_NR_120_GL_-_final.pdf HTTP/1.1,250
GET /verejnost/aktuality/foto/image?view=image&format=raw&type=orig&id=16557 HTTP/1.1,238


In [8]:
df.drop(df[df['Request'].str.contains('navbar')].index, inplace=True)
df.drop(df[df['Request'].str.contains('cron')].index, inplace=True)

In [9]:
import datetime

def getMonthNum(month):
    return {
        'Jan': 1,
        'Feb': 2,
        'Mar': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'Nov': 11,
        'Dec': 12
    }.get(month, -1) 

def parseDateToUnix(dat):
    day = dat[1:3]
    month = getMonthNum(dat[4:7])
    year = dat[8:12]
    hour = dat[13:15]
    minute = dat[16:18]
    second = dat[19:21]
    return datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second)).timestamp()

In [10]:
df['Unixtime']=df.apply(lambda row: parseDateToUnix(row['Datetime']), axis=1)

In [11]:
df = df.sort_values(by=["IP", "UserAgent", "Unixtime"])

In [12]:
# User identification

user = []
ip_before = "null"
agent_before = "null"
usid = 1
for ip,agent in zip(df['IP'],df['UserAgent']):
    if ip_before!="null":
        if ip_before==ip and agent_before==agent:
            user.append(usid)
        else:
            usid+=1
            user.append(usid)
    else:
        user.append(usid)
    ip_before = ip
    agent_before = agent

df['UserID'] = user

In [13]:
length = []
unxtm_before = -1
usr_before = -1
for unxtm,usr in zip(df['Unixtime'],df['UserID']):
    if usr_before!=-1:
        if usr==usr_before:
            unx_dif = unxtm-unxtm_before
            if unx_dif<=3600:
                length.append(unx_dif)
            else:
                length.append(None)
        else:
            length.append(None)
    unxtm_before = unxtm
    usr_before = usr

length.append(None)
df['Length'] = length

In [14]:
uppQ = df['Length'].quantile(0.75)
lowQ = df['Length'].quantile(0.25)
Q = uppQ + 1.5 * (uppQ-lowQ)

sttQ = []
usr_before = -1
length_before = -1
stQ = 1
for length,usr in zip(df['Length'],df['UserID']):
    if usr_before!=-1:
        if usr==usr_before:
            if length_before<=Q:
                sttQ.append(stQ)
            else:
                stQ+=1
                sttQ.append(stQ)
        else:
            stQ+=1
            sttQ.append(stQ)
    else:
        sttQ.append(stQ)
    usr_before = usr
    length_before = length

df['STT_Q'] = sttQ

In [15]:
# data transformation
def parseCategory(req):
    if(req=="GET / HTTP/1.1"): 
        return "home"
    strng = req.split("/") 
    if len(strng)>2:
        out = strng[1].replace(" HTTP","")
        if(out=="en"):
            return "home"
        else:
            return out
    

df['Category'] = df.apply(lambda row: parseCategory(row['Request']), axis=1)

In [16]:
req_tab = pd.crosstab(index=df['Category'], columns="count")
req_tab = req_tab.sort_values(by=["count"], ascending=False)
req_tab.head(15)

col_0,count
Category,
verejnost,6966
home,5947
fakulty-a-sucasti,1283
univerzita,1269
images,997
prijimacie-konanie,897
studium,833
media-a-marketing,246
administrator,227


In [17]:
def updateCategory(cat):
    if(cat=="verejnost" or cat=="home" or cat=="fakulty-a-sucasti" or cat=="univerzita" or cat=="images" 
       or cat=="prijimacie-konanie" or cat=="studium" or cat=="media-a-marketing" or cat=="administrator" or cat=="component"):
        return cat
    else:
        return "other"

df['Category'] = df.apply(lambda row: updateCategory(row['Category']), axis=1)

In [18]:
df.Category.unique()

array(['home', 'univerzita', 'other', 'fakulty-a-sucasti', 'studium',
       'verejnost', 'images', 'component', 'prijimacie-konanie',
       'media-a-marketing', 'administrator'], dtype=object)

In [19]:
req_tab = pd.crosstab(index=df['STT_Q'], columns="count")
req_tab = req_tab.sort_values(by=["count"], ascending=False)
req_tab.head(5)

col_0,count
STT_Q,
1803,204
4107,165
2129,165
435,132
2785,128


In [20]:
sess = df.sort_values('STT_Q').groupby('STT_Q')['Category'].apply(lambda df: df.reset_index(drop=True)).unstack()
sess

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,202,203
STT_Q,,,,,,,,,,,,,,,,,,,,,
1,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8002,other,prijimacie-konanie,fakulty-a-sucasti,other,other,fakulty-a-sucasti,other,fakulty-a-sucasti,fakulty-a-sucasti,fakulty-a-sucasti,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8003,home,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8004,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data analysis

In [21]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [23]:
items = df.Category.unique()

In [24]:
itemset = set(items)
encoded_vals = []
for index, row in sess.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]

ohe_df = pd.DataFrame(encoded_vals)

In [25]:
freq_items = apriori(ohe_df, min_support=0.01, use_colnames=True, verbose=1)
freq_items = freq_items.sort_values(by=["support"], ascending=False)
freq_items.head()

Processing 51 combinations | Sampling itemset size 3


,support,itemsets
9,0.545966,(home)
2,0.125781,(verejnost)
4,0.119910,(other)
6,0.111167,(univerzita)
8,0.102798,(fakulty-a-sucasti)


In [26]:
rules = association_rules(freq_items, metric="support", min_threshold=0.01)
rules = rules.sort_values(by=["support"], ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(home),(fakulty-a-sucasti),0.545966,0.102798,0.038346,0.070236,0.683240,-0.017778,0.964978
1,(fakulty-a-sucasti),(home),0.102798,0.545966,0.038346,0.373026,0.683240,-0.017778,0.724167
2,(home),(univerzita),0.545966,0.111167,0.032975,0.060398,0.543311,-0.027718,0.945968
3,(univerzita),(home),0.111167,0.545966,0.032975,0.296629,0.543311,-0.027718,0.645512
4,(home),(verejnost),0.545966,0.125781,0.031476,0.057653,0.458359,-0.037196,0.927704


In [27]:
rules = rules.sort_values(by=["lift"], ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(images),(verejnost),0.076817,0.125781,0.023982,0.312195,2.482060,0.014320,1.271028
7,(verejnost),(images),0.125781,0.076817,0.023982,0.190665,2.482060,0.014320,1.140669
21,(studium),(images),0.072571,0.076817,0.011491,0.158348,2.061352,0.005917,1.096869
20,(images),(studium),0.076817,0.072571,0.011491,0.149593,2.061352,0.005917,1.090572
16,(prijimacie-konanie),(fakulty-a-sucasti),0.076443,0.102798,0.012990,0.169935,1.653094,0.005132,1.080881
